In [1]:
import pandas as pd
import getpass
from sqlalchemy import create_engine
from sqlalchemy.exc import ResourceClosedError
import MySQLdb 
check = False

In [2]:
# Uzdevums:
# Nokopējiet faila make_db.sql, kas atrodāas githubā, saturu, 
# atveriet mysql shell caur termināli 
# un ielīmējiet/izpildiet make_db.sql esošo kodu

# Šī ir MySQL parauga datubāze, kura ir aprakstīta šeit:
# https://www.mysqltutorial.org/mysql-sample-database.aspx

In [3]:

class MySQLConnection:
    def __init__(self,user,hostname,db):
        self.user = str(user)
        self.hostname = str(hostname)
        self.db = str(db)
        pword = getpass.getpass("Enter password for user {}".format(user))
        # self.cnx = mysql.connector.connect(host=self.hostname,user=self.user, password=pword,database=self.db)
        self.engine = create_engine("mysql+mysqlconnector://{}:{}@{}/{}".format(user,pword,hostname,db))
        self.cnx = self.engine.connect()
    
    def write_to_db(self, df, table_name):
        try:
            df.to_sql(table_name,con = self.cnx,if_exists='fail',index=False)
        except Exception as e:
            print("\n SQL Write error with: ")
            print(df,"\n", e)
    
    def query(self,q):
        try:
            res = pd.read_sql(q,self.cnx)
            return res
        except ResourceClosedError:
            print("Query success, nothing was returned")

In [8]:
sql = MySQLConnection('ubuntu','localhost','dati')

Enter password for user ubuntu········


In [9]:
sql.query("select * from orders")

,order_id,order_date,amount,customer_id
0,1,07/04/1776,$234.56,1
1,2,03/14/1760,$78.50,3
2,3,05/23/1784,$124.00,2
3,4,09/03/1790,$65.50,3
4,5,07/21/1795,$25.50,10
5,6,11/27/1787,$14.40,9


In [10]:
sql.query("select * from customers")

,customer_id,first_name,last_name,email,address,city,state,zipcode
0,1,George,Washington,gwashington@usa.gov,3200 Mt Vernon Hwy,Mount Vernon,VA,22121
1,2,John,Adams,jadams@usa.gov,1250 Hancock St,Quincy,MA,2169
2,3,Thomas,Jefferson,tjefferson@usa.gov,931 Thomas Jefferson Pkwy,Charlottesville,VA,22902
3,4,James,Madison,jmadison@usa.gov,11350 Constitution Hwy,Orange,VA,22960
4,5,James,Monroe,jmonroe@usa.gov,2050 James Monroe Parkway,Charlottesville,VA,22902


In [12]:
# 4. Katrā pilsētā ir tieši viens kurjers, cik piegāžu būs jāveic katram kurjeram?
# sql.query("SELECT * from orders")

# No sākuma, tabulas apvienosim un rezultātu ievietosim jaunā tabulā
q = """
create table join_rezultats as 
select 
    order_id, 
    city
from 
    customers join orders on 
    customers.customer_id = orders.order_id
"""
sql.query(q)

Query success, nothing was returned


In [13]:
sql.query("select * from join_rezultats")

,order_id,city
0,1,Mount Vernon
1,2,Quincy
2,3,Charlottesville
3,4,Orange
4,5,Charlottesville


In [ ]:
# Mums bija jāizdara vairāki selekti, lai izveidotu jaunas 'tabulas'. 
# Tik, cik ir pilsētu.
# Un tad jāsaskaita cik ierakstu / piegāžu ir katrā šajā 'tabulā'

# Šo darbību var automatizēt izmantojot GROUP BY

# GROUP BY  
GROUP BY veic grupēšanu pa kādas kolonnas unikālajām vērtībām un aprēķina kādu 'kopsavilkuma' funkciju katrai šai grupai.  

Piemēram, ja mums ir tabula, kurā ir 15 rindiņas un  3 kolonnas a, b un c, kur c kolonnā ir divas unikālas vērtības, tad GROUP BY pēc c kolonnas, mums izveidos divas grupas. Viena katrai unikālajai c vērtībai. Uz šīm grupām tad obligāti ir jāuzliek kāda kopsavilkuma funkcija.  

Lietojamās kopsavilkuma funkcijas ir sekojošās:  
* COUNT  
* SUM  
* AVG  
* MIN  
* MAX  
Atkarībā no specifiskās SQL versijas, ko lietojat, šajā sarakstā var arī būt vēl kādas funkcijas

In [15]:
q2 = """
select 
    city,
    count(*) as piegazu_skaits
from 
    join_rezultats
group by
    city
"""
sql.query(q2)

,city,piegazu_skaits
0,Charlottesville,2
1,Mount Vernon,1
2,Orange,1
3,Quincy,1


In [4]:
# Uzdevums:
# 1. Resetojiet kerneli, lai izveidotu jaunu sql objektu, iestādiet datubāzi uz classicmodels. 
# Visi sekojošie uzdevumi būs ar datubāzi classicmodels un ir jāpēta datubāzes modelis, lai tos izdomātu.

sql = MySQLConnection('ubuntu','localhost','classicmodels')

# 2. Izpētiet datubāzes classicshēmuSaskaitiet cik piegāžu ir katrā pilsētā, 
# izmantojot tabulas customers un orders no datubāzes classicmodels.

Enter password for user ubuntu········


In [ ]:
# Uzdevums:
# Saskaitiet cik ir produktu līniju un cik katrā produktu līnijā ir unikālu produktu.

In [ ]:
# Uzdevums:
# Aprēķiniet cik daudz ienākumu ir no katras produktu līnijas


In [ ]:
#Uzdevums:
# Aprēķiniet cik ir ienākumu no katras pilsētas

# Vairāku līmeņu SELECT  
Bieži rodās situācijas, kur mēs nevaram nepieciešamos datus apvienojot tikai divas tabulas. 

In [12]:
q3 = """
select 
    priceEach,
    pilseta
from 
    orderdetails join 
    (
    select 
        customers.city as pilseta, 
        orders.ordernumber as ordernum
    from 
        customers join orders on
        customers.customernumber = orders.customerNumber
    ) pasutijumi on 
    orderdetails.ordernumber = pasutijumi.ordernum
"""
# test_q = """
# select 
#     customers.city as pilseta, 
#     orders.ordernumber as ordernum
# from 
#     customers join orders on
#     customers.customernumber = orders.customerNumber
# """
sql.query(q3)

,priceEach,pilseta
0,120.71,Nantes
1,114.84,Nantes
2,117.26,Nantes
3,43.27,Nantes
4,105.86,Nantes
...,...,...
2991,123.51,Auckland
2992,51.48,Auckland
2993,104.81,Auckland
2994,75.41,Auckland


In [14]:
# Tagad atliek tikai sagrupēt un priceeach kolonnas vērtības katrā izveidotajā grupā.

q3_grouped = """
select 
    sum(priceeach) as ienakumi,
    pilseta
from 
    orderdetails join 
    (
    select 
        customers.city as pilseta, 
        orders.ordernumber as ordernum
    from 
        customers join orders on
        customers.customernumber = orders.customerNumber
    ) pasutijumi on 
    orderdetails.ordernumber = pasutijumi.ordernum
group by 
    pilseta
"""
sql.query(q3_grouped)

,ienakumi,pilseta
0,2906.65,Allentown
1,10576.15,Auckland
2,1776.13,Barcelona
3,3793.04,Bergamo
4,2857.73,Bergen
...,...,...
72,2153.72,Vancouver
73,1613.88,Versailles
74,2426.92,Wellington
75,2245.94,White Plains


Vispārīga sintakse iekšējo joinu veidošanai:  


In [ ]:
"""
SELECT
    kolonna1,
    kolonna2
FROM 
    areja_joina_tabula <join tips> join 
    (
        
    <
    SELECT statements un/vai joins, 
    kas atgriež tabulu, kura satur indeksu, 
    ko var sasaistīt ar tabulu areja_join_tabula 
    >

    ) iekseja_join_tabula on 
    areja_join_tabula.indeksa_kolonna = iekseja_join_tabula.indeksa_kolonna_velams_nosaukta_citadi
"""

Praktiski padomi strādājot ar vairākiem joiniem:  
* Izmantojiet aliasus
* Vienmēr iztestējiet iekšējo joinu kā pirmo. Pārbaudiet vai kolonnas ir nosauktas, vai vērtības šķiet saprātīgas.


In [ ]:
# Uzdevums: Saskaitiet cik pasūtījumu ir katrā pilsētā.
# Padoms: to var izdarīt ļoti minimāli modificējot q3_grouped vaicājuma kodu